<a href="https://colab.research.google.com/github/meetarora10/veersa-hack/blob/main/model_summarise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install transformers fpdf2 torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 6.6 MB/s eta 0:00:00


In [1]:
from transformers import pipeline
from fpdf import FPDF
import textwrap
import os
import requests

/usr/local/lib/python3.11/dist-packages/fpdf/__init__.py:40: UserWarning: You have both PyFPDF & fpdf2 installed. Both packages cannot be installed at the same time as they share the same module namespace. To only keep fpdf2, run: pip uninstall --yes pypdf && pip install --upgrade fpdf2
  warnings.warn(


In [5]:

# 1. Load summarization pipeline
summarizer = pipeline("summarization", model="philschmid/bart-large-cnn-samsum")


# 2. Split chat text into chunks of 500 words
def chunk_text(text, max_words=500):
    words = text.split()
    return [' '.join(words[i:i + max_words]) for i in range(0, len(words), max_words)]

# 3. Summarize each chunk
def summarize_long_chat(long_chat_text):
    chunks = chunk_text(long_chat_text, max_words=500)
    summaries = []

    for i, chunk in enumerate(chunks):
        print(f"Summarizing chunk {i+1}/{len(chunks)}...")
        summary = summarizer(chunk, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
        summaries.append(summary)

    return ' '.join(summaries)


# Download DejaVuSans if not present
font_url = "https://github.com/dejavu-fonts/dejavu-fonts/blob/master/ttf/DejaVuSans.ttf?raw=true"
font_path = "/usr/share/fonts/truetype/dejavu/DejaVuSans.ttf"

if not os.path.exists(font_path):
    os.makedirs(os.path.dirname(font_path), exist_ok=True)
    with open(font_path, "wb") as f:
        f.write(requests.get(font_url).content)

def create_pdf(summary_text, full_chat_text=None, filename="chat_summary.pdf"):
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    # Add Unicode font
    pdf.add_font('DejaVu', '', font_path, uni=True)
    pdf.set_font("DejaVu", size=14)
    pdf.cell(0, 10, txt="Doctor-Patient Chat Summary", ln=True, align="C")
    pdf.ln(10)

    # Summary
    pdf.set_font("DejaVu", size=12)
    pdf.cell(0, 10, txt="Summary:", ln=True)
    pdf.multi_cell(0, 10, summary_text)

    # Full chat
    if full_chat_text:
        pdf.add_page()
        pdf.cell(0, 10, txt="Full Chat Transcript:", ln=True)
        pdf.multi_cell(0, 10, full_chat_text)

    pdf.output(filename)
    print(f"PDF saved as {filename}")



# Example usage:
# long_chat_text = "Your long chat text here..."
# summary = summarize_long_chat(long_chat_text)
# create_pdf(summary_text=summary, full_chat_text=long_chat_text)


Device set to use cpu


In [6]:
long_chat_text = """
Doctor: Hello! How are you feeling today?
Patient: Not too great, doctor. I've been feeling feverish and weak since last night.
Doctor: I’m sorry to hear that. Do you know what your temperature was?
Patient: Yes, it was around 101.2°F when I checked this morning.
Doctor: Any other symptoms apart from the fever?
Patient: My throat is sore and I’ve been feeling a bit nauseous too.
Doctor: Do you have a cough or any difficulty breathing?
Patient: No cough yet, and my breathing is normal.
Doctor: Have you traveled anywhere recently or been in contact with anyone sick?
Patient: No recent travel, but one of my colleagues was unwell last week.
Doctor: That could be a possible source. Have you taken any medication so far?
Patient: I took a paracetamol tablet this morning, it brought the fever down slightly.
Doctor: How’s your appetite?
Patient: Not great. I skipped breakfast because I was feeling nauseous.
Doctor: I see. And any stomach pain or loose motion?
Patient: A little discomfort, but no diarrhea.
Doctor: What’s your hydration level? Are you drinking enough fluids?
Patient: Trying to. I’ve had two glasses of water and some soup so far.
Doctor: That’s good. Keep sipping water frequently. Have you had any similar episodes before?
Patient: I do get mild seasonal infections around this time, but not usually with nausea.
Doctor: Are you allergic to any medications?
Patient: Not that I’m aware of.
Doctor: Any ongoing conditions like diabetes, hypertension, or asthma?
Patient: No, I’m otherwise healthy.
Doctor: That’s good. I’ll suggest continuing paracetamol every 6 hours if the fever persists.
Patient: Okay, and should I eat something before taking it?
Doctor: Yes, preferably after food or with a light snack to avoid gastric issues.
Patient: Got it. Should I get tested for anything?
Doctor: If the fever continues for more than 2 days, we’ll do a blood test and maybe a COVID test too.
Patient: Alright, I’ll monitor closely. Is it safe to rest at home?
Doctor: Yes, but if you experience high-grade fever, shortness of breath, or persistent vomiting, visit a clinic immediately.
Patient: Sure. Thank you for the guidance, doctor.
Doctor: You're welcome. Take rest, eat light, stay hydrated, and don’t worry. I’ll check back tomorrow.
Patient: Thanks a lot!
Doctor: Take care.
"""


In [7]:
# Step 1: Summarize
summary = summarize_long_chat(long_chat_text)

# Step 2: Save to PDF
create_pdf(summary_text=summary, full_chat_text=long_chat_text)

Summarizing chunk 1/1...
PDF saved as chat_summary.pdf


<ipython-input-5-2de6037eca35>:46: DeprecationWarning: "uni" parameter is deprecated since v2.5.1, unused and will soon be removed
  pdf.add_font('DejaVu', '', font_path, uni=True)
<ipython-input-5-2de6037eca35>:48: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt="Doctor-Patient Chat Summary", ln=True, align="C")
<ipython-input-5-2de6037eca35>:48: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt="Doctor-Patient Chat Summary", ln=True, align="C")
<ipython-input-5-2de6037eca35>:53: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(0, 10, txt="Summary:", ln=True)
<ipython-input-5-2de6037eca35>:53: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(0, 10, txt="Summary:", ln=True)
<ipython-input-5-2de6037eca35>:59: Deprec

In [22]:
!apt-get install -y fonts-dejavu


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-dejavu is already the newest version (2.37-2build1).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
